In [8]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from sklearn.utils.extmath import randomized_svd
 
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.animation import FuncAnimation
from dotenv import dotenv_values 
from moviepy.editor import VideoClip
from moviepy.video.io.bindings import mplfig_to_npimage
import scipy.io
from scipy.signal import cont2discrete
import pyvista as pv
import h5py
import mat73
import dask.array as da
from dask.distributed import Client, LocalCluster
config = dotenv_values(".env")


In [9]:
client = Client('tcp://127.0.0.1:8786')
client.cluster


In [33]:
# Test difference between dask svd and svd_compressed methods
def svd_compressed(X, n_components):
    U, S, V = da.linalg.svd_compressed(X, k=n_components)
    U, S, V = da.compute(U, S, V)
    return U, S, V
def svd_dask(X, n_components):
    U, S, V = da.linalg.svd(X)
    U, S, V = da.compute(U, S, V)

    return U, S, V


In [ ]:
# Make a well-structured matrix that we can visually check to see if the results are the same
X = np.array([[1, 2, 3,4], [4, 4, 5, 6], [4, 7, 8, 9]])
# Make a dask array
X_dask = da.from_array(X, chunks=(1, -1))
# Compare the two methods
U, S, Vh = svd_compressed(X_dask, 2)
U2, S2, Vh2 = svd_dask(X_dask, 2)
# Check the results
# # Check if we can reconstruct the original matrix
# print(U @ np.diag(S) @ Vh)
# # Check if we can reconstruct the original matrix
# print(U2 @ np.diag(S2) @ Vh2)
# # Print individual svd matrices
print(U)
print(S)
print(Vh)
# Print individual svd matrices
print(U2)
print(S2)
print(Vh2)
# Check if the results are the same



[[ 0.2963798  -0.4984734 ]
 [ 0.52691826  0.79677054]
 [ 0.79656523 -0.34158596]]
[18.17434906  1.38964533]
[[ 0.30759362  0.45538848  0.54451758  0.63364667]
 [ 0.95151253 -0.144617   -0.17576798 -0.20691896]]
[[ 0.2963798  -0.4984734  -0.81466759]
 [ 0.52691826  0.79677054 -0.2958274 ]
 [ 0.79656523 -0.34158596  0.49880143]]
[18.17434906  1.38964533  0.87288151]
[[ 0.30759362  0.45538848  0.54451758  0.63364667]
 [ 0.95151253 -0.144617   -0.17576798 -0.20691896]
 [-0.00317507  0.77784355  0.07706854 -0.62370648]]


ValueError: operands could not be broadcast together with shapes (3,2) (3,3) 

In [ ]:

####### # Set up hyperparameters for all ROMs  #######
n_timesteps = 1000 # Number of timesteps to pull from each episode
dt = 0.01
startTrial = 39 # Episode to start training on
# Training Trials 
train_trials = [1]#[0, 1, 4, 5, 7, 8, 11, 14, 16, 18, 20, 22, 24, 28, 31, 32, 33, 35, 37, 39]#[0,1,3,4,5,6,7,8,11,12,13,14,15,16,18,19,20,22,23,24,26,27,28,29,30,31,32,33,35,36,37,39] #[39]
n_train = len(train_trials) # Number of training trials
# n_train = 1 # Number of training episodes
ns_ROM = np.arange(2,202,2) # Dimensions of ROM state space



####### Set up filepaths #######
# Setup filepaths for reading data 
filepath = config["currentDirectory"] + "data/archivedDataSets/ContiguousAssembly/"
romDir = config["currentDirectory"] + "data/archivedDataSets/ContiguousAssembly/ROMs/"

####### Load in data #######
print("Loading in data...")
# Read in all training data 
data = h5py.File(filepath+ "FreqSweepDataset.hdf5", 'r')
print(list(data.keys()))
X_fom = da.from_array(data["stateData"], chunks=(1024, 4096,1))
X_fom = X_fom[:,:,train_trials]
U_fom = da.from_array(data["inputData"], chunks=(1024, 4096,1))
U_fom = U_fom[:,:,train_trials]
Y_fom = da.from_array(data["outputData"], chunks=(1024, 4096,1))
Y_fom = Y_fom[:,:,train_trials]
x0 = data["stateData"][:,0,train_trials[0]]
y0 = data["outputData"][:,0,train_trials[0]]
n = X_fom.shape[0]
l = U_fom.shape[0]
m = Y_fom.shape[0]
# # Collect initial condition offsets for each episode and center each episode about that initial condition
X0 = da.zeros((n,n_train))
Y0 = da.zeros((m,n_train))
for i in range(n_train):
    X0[:,i] = X_fom[:,0,i]
    Y0[:,i] = Y_fom[:,0,i]
print("Done loading in data.")
X_fom = X_fom - X0[:,0].reshape(-1,1,1)
Y_fom = Y_fom - Y0[:,0].reshape(-1,1,1)


# Get length of robot from min and max of x coords
xzCoords = Y0[:,0].reshape(-1,2)

x_min = np.min(xzCoords[:,0])
x_max = np.max(xzCoords[:,0])
length = x_max - x_min
print("Length of robot: ", length.compute())
# Get z min and max
z_min = np.min(xzCoords[:,1])
z_max = np.max(xzCoords[:,1])

In [ ]:
# Initialize data matrices for training and testing
X_train = da.zeros((n,(n_timesteps-1)*n_train), chunks=(1024, (n_timesteps-1)*n_train))
Xprime_train = da.zeros((n,(n_timesteps-1)*n_train), chunks=(1024, (n_timesteps-1)*n_train))
Upsilon_train = da.zeros((l,(n_timesteps-1)*n_train), chunks=(1024, (n_timesteps-1)*n_train))
Y_train = da.zeros((m,(n_timesteps-1)*n_train), chunks=(1024, (n_timesteps-1)*n_train))
print("got here 2")
# Load in data from training episodes
for i in range(n_train):
    X_train[:,i*(n_timesteps-1):(i+1)*(n_timesteps-1)] = X_fom[:,:-1,i]
    Xprime_train[:,i*(n_timesteps-1):(i+1)*(n_timesteps-1)] = X_fom[:,1:,i]
    Upsilon_train[:,i*(n_timesteps-1):(i+1)*(n_timesteps-1)] = U_fom[:,:-1,i]
    Y_train[:,i*(n_timesteps-1):(i+1)*(n_timesteps-1)] = Y_fom[:,:-1,i]
print("got here 3")
# Form snapshot matrices for DMDc
Omega = da.concatenate([X_train,Upsilon_train],axis=0)
print("got here 4")
# # Precompute truncated SVD of Omega
p_dmd = (n_timesteps-1)*n_train# dimension of reduced input space


In [ ]:
n_ROM = 2
# Decompose snapshot matrix using truncated SVD
r_dmd = n_ROM
p_dmd = np.min([1000,(n_timesteps-1)*n_train])
#(n_timesteps-1)*n_train# dimension of reduced input space

# print("got here 5")
U_tilde,Sigma_tilde,Vh_tilde = da.linalg.svd(Omega)
# Truncate SVD matrices to form rank p_dmd approximation
U_tilde = U_tilde[:,0:p_dmd]
Sigma_tilde = Sigma_tilde[0:p_dmd]
Vh_tilde = Vh_tilde[0:p_dmd,:]
# Vh_tilde = da.conj(da.transpose(V_tilde))
Sigma_tilde = da.diag(Sigma_tilde) # convert Sigma_tilde to diagonal matrix
U_hat, Sigma_hat, Vh_hat = da.linalg.svd(Xprime_train)
# Truncate SVD matrices to form rank r_dmd approximation
U_hat = U_hat[:,0:r_dmd]
Sigma_hat = Sigma_hat[0:r_dmd]
Vh_hat = Vh_hat[0:p_dmd,:]
Sigma_hat = da.diag(Sigma_hat) # convert Sigma_hat to diagonal matrix
# print("got here 6")
# Compute system state evolution matrices
U_tilde_1 = U_tilde[0:n,:]
U_tilde_2 = U_tilde[n:,:]
A_dmdc = (U_hat.conj().T)@Xprime_train@(Vh_tilde.conj().T)@(da.linalg.inv(Sigma_tilde))@(U_tilde_1.conj().T)@U_hat
B_dmdc = (U_hat.conj().T)@Xprime_train@(Vh_tilde.conj().T)@(da.linalg.inv(Sigma_tilde))@(U_tilde_2.conj().T)

# print("got here 7")
ys = Y_train[:,0:250]

# compute approximate pseudoinverse of data matrix using SVD
# X_train = X_train.rechunk((2000,X_train.shape[1])) # Rechunk along one dimension
U_X_train, S_X_train, Vh_X_train = da.linalg.svd(X_train[:,0:250])
X_PI = da.matmul(da.transpose(Vh_X_train),da.matmul(da.diag(da.reciprocal(S_X_train)),da.transpose(U_X_train)))

# print("got here 8")

# # X_PI = np.linalg.pinv(X_train[:,0:10])
C_dmdc = ys@X_PI
C_dmdc = C_dmdc@U_hat

# print("got here 9")
# Execute computations for dmdc system matrices
print("Computing DMDc system matrices for n_ROM = ", n_ROM)
A_dmdc, B_dmdc, C_dmdc,U_hat = da.compute(A_dmdc, B_dmdc, C_dmdc,U_hat)
# A_dmdc, B_dmdc = da.compute(A_dmdc, B_dmdc)
# A_dmdc = A_dmdc.compute()
# print("got here 10")
# B_dmdc = B_dmdc.compute()
# print("got here 11")
# C_dmdc = C_dmdc.compute()
# print("got here 12")
# U_hat = U_hat.compute()
# print("got here 13")
basis_dmdc = U_hat[:,0:n_ROM]